### Selected Word Group In Youtube Sentence

In [1]:
import os
import re
import pandas as pd
import numpy as np
import glob
from functools import reduce
from pathlib import Path
import shutil
from itertools import islice

In [2]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Arabic"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# native word select for Part 2 
word_start = 0  # 0  # native word start index
word_end = 200 # 28  # native word end index

# source path
source_path = "/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Deployment2/Result/Deploy2 Result Manuel/Turkish"

# parameter
file = "Turkish_28_Word_Threegram_Manuel"  # without xlsx extention
sheets = "Sheet1"  # 2 gram target, 2 gram hybrid
file_ext = "28_Word_Threegram"
first_column = 0  # start column location number
end_column = 1
time_shift = 0.6  # 0.6
sample_num = 6  # 6

# youtube sentence condition
sent_time_start = False  # True, False for activating start time condition
sent_start_time = 700  # sn
random_sample = True  # True, False

In [3]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Result/7-Selected Word Group In Youtube Sentence/{lang_folder.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

In [4]:
def word_group_youtube(df, search_list, target_column, sample_num):
    '''
    word_group_youtube(df_youtube_sentence, search_list, "sentence", 6)
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        try:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].sample(sample_num)
        except:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].head(sample_num)
        #df_result = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)]  # sentence length part
        #df_result.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True)
        #df_select = df_result.head(sample_num)
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [5]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    df.reset_index(drop=True, inplace=True)
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [6]:
def word_count_result(df, column_list, set_condition=False): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        if set_condition:
            var_list = set(df[f"{i}"].dropna().tolist())
        else:
            var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    df_word_count.sort_values("word_count", ascending=False, inplace=True)
    df_word_count.reset_index(inplace=True, drop=True)
    
    return  df_word_count

In [7]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
987924,karneleme,5
987925,karnaya,5
987926,dörtlümüzün,5
987927,karnavalınız,5


In [8]:
df_word_select = df_word_all.iloc[word_start:word_end,]
df_word_select

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
195,saat,399989
196,onunla,399330
197,yapıyorsun,398274
198,neler,397377


In [9]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:00.294,00:00:03.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU
1,00:00:03.375,00:00:06.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU
2,00:00:06.462,00:00:09.462,wwwsebederorg,fI1BI4d5KFU
3,00:00:10.267,00:00:11.394,mumu,fI1BI4d5KFU
4,00:00:11.835,00:00:14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU
...,...,...,...,...
3934202,00:10:00.240,00:10:07.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0
3934203,00:10:07.994,00:10:13.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0
3934204,00:10:14.745,00:10:17.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0
3934205,00:10:18.047,00:10:22.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0


In [10]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [11]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,0.294,3.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU
1,3.375,6.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU
2,6.462,9.462,wwwsebederorg,fI1BI4d5KFU
3,10.267,11.394,mumu,fI1BI4d5KFU
4,11.835,14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU
...,...,...,...,...
3934202,600.240,607.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0
3934203,607.994,613.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0
3934204,614.745,617.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0
3934205,618.047,622.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0


In [12]:
if sent_time_start:
    df_youtube_select = df_youtube_sentence[df_youtube_sentence["start_time"] <= sent_start_time]
    df_youtube_select.reset_index(drop=True, inplace=True)
else:
    df_youtube_select = df_youtube_sentence

if random_sample:
    df_youtube_select = df_youtube_select.sample(len(df_youtube_select))
    df_youtube_select.reset_index(drop=True, inplace=True)
else:
    pass

df_youtube_select

,start_time,end_time,sentence,video_id
0,3964.649,3966.299,ben diyeceğimi dedim,jluoByJbQgg
1,499.139,501.637,arazi kazandırma metodu tek çözüm olmamalı,klzFOf6Yf6M
2,2391.110,2396.240,bu yılın yedisi harp müddeti diğer yedisi ise ...,wZw4WSieZGc
3,5621.772,5623.326,ne olur hemen gelin lütfen,27kv5lOOTGc
4,6889.393,6891.758,ama benim olduğumu bilmesin sedat,TAHq3DYE_Rw
...,...,...,...,...
3934202,166.867,167.367,beğeneceğinizden eminim,dpM8oYasFuE
3934203,1483.040,1484.220,artık beni duymuyorsun,zJMggnPkep8
3934204,3365.403,3366.203,hadi hadi,5NaAEyaQJrA
3934205,3918.183,3918.983,evet,9J7s6l7Ob4Q


In [13]:
df_word_group_select = pd.read_excel(f"{source_path}/{file}.xlsx", sheet_name= f"{sheets}")
df_word_group_select

,threegram,frequency
0,bir şey daha,26750
1,yok bir şey,22514
2,o da ne,18569
3,evet ben de,16980
4,o kadar çok,12920
...,...,...
62,evet evet bu,492
63,ne için bu,484
64,evet ne kadar,475
65,bu da var,446


In [ ]:
# Option 
# Skip if it doesn't need
df_remove = pd.read_excel(f"")
df_remove

In [ ]:
remove_videoid_list = list(set(df_remove["video_id"]))
#remove_videoid_list

In [ ]:
len(remove_videoid_list)

In [ ]:
# Option
df_youtube_select = df_youtube_select[~df_youtube_select["video_id"].isin(remove_videoid_list)]
df_youtube_select

In [14]:
search_list = []
for i in range(len(df_word_group_select)):
    for j in df_word_group_select.columns[first_column:end_column]:
        string = df_word_group_select.loc[i,j]
        
        if pd.isnull(string) == False and string != 'nan':
            search_list.append(string.strip())
            search_list = list(set(search_list))  # for unique value
        else:
            pass

In [15]:
len(search_list)

67

In [16]:
df_search_result = word_group_youtube(df_youtube_select, search_list, "sentence", sample_num)
df_search_result

,search_string,start_time,end_time,sentence,video_id
0,bu kadar değil,5256.980,5258.660,çapınız bu kadar değil mi,5n_ZSdhY20k
1,bu kadar değil,2327.340,2330.420,ama elbetteki nefis sarmalarımın sırrı bu kada...,8kZk1TgOXfQ
2,bu kadar değil,6323.552,6325.060,hayır bu kadar değil,EyxjFfK2GWA
3,bu kadar değil,5493.487,5494.894,kızım bu kadar değil ya,TSFUMTGQUWw
4,bu kadar değil,2841.922,2843.539,sayıları bu kadar değil,3aUTbWogu4E
...,...,...,...,...,...
363,evet evet bu,1515.227,1519.058,evet evet bu iki inatçı keçinin inadı kırılana...,x6aEnqpTUBo
364,evet evet bu,5120.570,5123.730,evet evet bu gördüğünüz ürün sayesinde,72dK6BETM6Q
365,ben ve sen,72.240,76.840,gel dans edelim sen ve ben el ele baş başa ben...,DpdZLavMyRk
366,ben ve sen,17402.070,17405.958,o şarap ister ki ben ve sen demeyip zikirde ol...,lHNtdUcq2_A


In [17]:
df_word_group_time_loc_result = word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id
0,bu kadar değil,5257.450400,5258.525600,çapınız bu kadar değil mi,5n_ZSdhY20k
1,bu kadar değil,2329.548302,2330.420000,ama elbetteki nefis sarmalarımın sırrı bu kada...,8kZk1TgOXfQ
2,bu kadar değil,6323.929000,6325.060000,hayır bu kadar değil,EyxjFfK2GWA
3,bu kadar değil,5493.792870,5494.771652,kızım bu kadar değil ya,TSFUMTGQUWw
4,bu kadar değil,2842.484435,2843.539000,sayıları bu kadar değil,3aUTbWogu4E
...,...,...,...,...,...
363,evet evet bu,1515.227000,1516.057050,evet evet bu iki inatçı keçinin inadı kırılana...,x6aEnqpTUBo
364,evet evet bu,5120.570000,5121.651053,evet evet bu gördüğünüz ürün sayesinde,72dK6BETM6Q
365,ben ve sen,75.885283,76.840000,gel dans edelim sen ve ben el ele baş başa ben...,DpdZLavMyRk
366,ben ve sen,17403.314160,17404.247280,o şarap ister ki ben ve sen demeyip zikirde ol...,lHNtdUcq2_A


In [18]:
df_word_group_time_loc_result.start_time = df_word_group_time_loc_result.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc_result.end_time = df_word_group_time_loc_result.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id
0,bu kadar değil,5256.850400,5259.125600,çapınız bu kadar değil mi,5n_ZSdhY20k
1,bu kadar değil,2328.948302,2331.020000,ama elbetteki nefis sarmalarımın sırrı bu kada...,8kZk1TgOXfQ
2,bu kadar değil,6323.329000,6325.660000,hayır bu kadar değil,EyxjFfK2GWA
3,bu kadar değil,5493.192870,5495.371652,kızım bu kadar değil ya,TSFUMTGQUWw
4,bu kadar değil,2841.884435,2844.139000,sayıları bu kadar değil,3aUTbWogu4E
...,...,...,...,...,...
363,evet evet bu,1514.627000,1516.657050,evet evet bu iki inatçı keçinin inadı kırılana...,x6aEnqpTUBo
364,evet evet bu,5119.970000,5122.251053,evet evet bu gördüğünüz ürün sayesinde,72dK6BETM6Q
365,ben ve sen,75.285283,77.440000,gel dans edelim sen ve ben el ele baş başa ben...,DpdZLavMyRk
366,ben ve sen,17402.714160,17404.847280,o şarap ister ki ben ve sen demeyip zikirde ol...,lHNtdUcq2_A


In [19]:
df_word_group_time_loc_result.start_time = df_word_group_time_loc_result.start_time.apply(lambda x: round(x))
df_word_group_time_loc_result.end_time = df_word_group_time_loc_result.end_time.apply(lambda x: round(x))
df_word_group_time_loc_result 

,search_string,start_time,end_time,sentence,video_id
0,bu kadar değil,5257,5259,çapınız bu kadar değil mi,5n_ZSdhY20k
1,bu kadar değil,2329,2331,ama elbetteki nefis sarmalarımın sırrı bu kada...,8kZk1TgOXfQ
2,bu kadar değil,6323,6326,hayır bu kadar değil,EyxjFfK2GWA
3,bu kadar değil,5493,5495,kızım bu kadar değil ya,TSFUMTGQUWw
4,bu kadar değil,2842,2844,sayıları bu kadar değil,3aUTbWogu4E
...,...,...,...,...,...
363,evet evet bu,1515,1517,evet evet bu iki inatçı keçinin inadı kırılana...,x6aEnqpTUBo
364,evet evet bu,5120,5122,evet evet bu gördüğünüz ürün sayesinde,72dK6BETM6Q
365,ben ve sen,75,77,gel dans edelim sen ve ben el ele baş başa ben...,DpdZLavMyRk
366,ben ve sen,17403,17405,o şarap ister ki ben ve sen demeyip zikirde ol...,lHNtdUcq2_A


In [20]:
df_word_group_time_loc_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_result['video_id'].map(str)+"&t="+df_word_group_time_loc_result['start_time'].map(str)+"s"
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,bu kadar değil,5257,5259,çapınız bu kadar değil mi,5n_ZSdhY20k,https://www.youtube.com/watch?v=5n_ZSdhY20k&t=...
1,bu kadar değil,2329,2331,ama elbetteki nefis sarmalarımın sırrı bu kada...,8kZk1TgOXfQ,https://www.youtube.com/watch?v=8kZk1TgOXfQ&t=...
2,bu kadar değil,6323,6326,hayır bu kadar değil,EyxjFfK2GWA,https://www.youtube.com/watch?v=EyxjFfK2GWA&t=...
3,bu kadar değil,5493,5495,kızım bu kadar değil ya,TSFUMTGQUWw,https://www.youtube.com/watch?v=TSFUMTGQUWw&t=...
4,bu kadar değil,2842,2844,sayıları bu kadar değil,3aUTbWogu4E,https://www.youtube.com/watch?v=3aUTbWogu4E&t=...
...,...,...,...,...,...,...
363,evet evet bu,1515,1517,evet evet bu iki inatçı keçinin inadı kırılana...,x6aEnqpTUBo,https://www.youtube.com/watch?v=x6aEnqpTUBo&t=...
364,evet evet bu,5120,5122,evet evet bu gördüğünüz ürün sayesinde,72dK6BETM6Q,https://www.youtube.com/watch?v=72dK6BETM6Q&t=...
365,ben ve sen,75,77,gel dans edelim sen ve ben el ele baş başa ben...,DpdZLavMyRk,https://www.youtube.com/watch?v=DpdZLavMyRk&t=75s
366,ben ve sen,17403,17405,o şarap ister ki ben ve sen demeyip zikirde ol...,lHNtdUcq2_A,https://www.youtube.com/watch?v=lHNtdUcq2_A&t=...


In [21]:
search_result_num = df_word_group_time_loc_result["search_string"].nunique()
search_result_num

66

In [22]:
search_result_set = set(df_word_group_time_loc_result["search_string"])
search_all_set = set(search_list)

In [23]:
search_all_set.difference(search_result_set)

{'o ne için'}

In [24]:
df_result_word_manuel_count = word_count_result(df_word_group_time_loc_result, df_word_group_time_loc_result.columns[0:1], set_condition=True)
df_result_word_manuel_count

,word,word_count
0,bu,26
1,çok,17
2,o,17
3,kadar,15
4,evet,13
5,da,11
6,ne,10
7,de,10
8,değil,10
9,hayır,9


In [25]:
# # Option merge
# df_word_count_merge = pd.merge(df_result_word_manuel_count, df_word_select[["word"]], how="inner", on="word")
# df_word_count_merge.drop_duplicates(inplace=True)
# df_word_count_merge.sort_values(by="word_count", ascending=False, inplace=True)
# df_word_count_merge.reset_index(drop=True, inplace=True)
# df_word_count_merge

In [26]:
set_word = set(df_word_select["word"])
#set_count = set(df_word_count_merge["word"])  # Option merge
set_count = set(df_result_word_manuel_count["word"])

In [27]:
set_word.difference(set_count)

{'adam',
 'al',
 'anne',
 'artık',
 'asla',
 'aslında',
 'aynı',
 'baba',
 'bak',
 'bakalım',
 'başka',
 'belki',
 'bence',
 'beni',
 'benim',
 'benimle',
 'bile',
 'biliyor',
 'biliyorsun',
 'biliyorum',
 'bilmiyorum',
 'biraz',
 'biri',
 'birkaç',
 'birlikte',
 'biz',
 'bize',
 'bizi',
 'bizim',
 'bugün',
 'buna',
 'bunu',
 'bunun',
 'burada',
 'buraya',
 'böyle',
 'bütün',
 'büyük',
 'dakika',
 'demek',
 'devam',
 'değilim',
 'diye',
 'dostum',
 'doğru',
 'dur',
 'ederim',
 'efendim',
 'en',
 'et',
 'eğer',
 'fazla',
 'gece',
 'gel',
 'geldi',
 'geliyor',
 'gerek',
 'geri',
 'gerçek',
 'gerçekten',
 'gibi',
 'git',
 'göre',
 'gün',
 'güzel',
 'hadi',
 'hakkında',
 'hala',
 'harika',
 'haydi',
 'hemen',
 'her',
 'herkes',
 'hey',
 'hiç',
 'hiçbir',
 'iki',
 'ile',
 'ilk',
 'ister',
 'istiyorsun',
 'istiyorum',
 'iyi',
 'izin',
 'içinde',
 'iş',
 'işte',
 'kendi',
 'ki',
 'kimse',
 'kötü',
 'küçük',
 'kız',
 'lazım',
 'lütfen',
 'merhaba',
 'misin',
 'mu',
 'musun',
 'mü',
 'mısın',
 

In [28]:
df_word_group_time_loc_result.to_excel(f"{lang_folder.capitalize()}_{search_result_num}_{file_ext}_\
{sample_num}_Youtube_{time_shift}s_Timeshift_Result.xlsx", index=False) 

In [29]:
# df_result_word_manuel_count.to_excel(f"{lang_folder.capitalize()}_{search_result_num}_{file_ext}_\
# {sample_num}_Youtube_{time_shift}s_Timeshift_Result_Word_Count.xlsx", index=False) 

#### Copy Move And Delete

In [30]:
output_file = glob.glob(f"{lang_folder.capitalize()}_*_{sample_num}_Youtube_{time_shift}s_Timeshift_Result*.xlsx")
output_file

['Turkish_66_28_Word_Threegram_6_Youtube_0.6s_Timeshift_Result.xlsx']

In [ ]:
for y in output_file:
    source = y # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for z in output_file:
    try:
        os.remove(z)
    except:
        pass

#### Temp

In [ ]:
set_result = set(df_word_group_time_loc_result["search_string"].unique())
set_selected = set(search_list)

In [ ]:
df_not_in_result = pd.DataFrame(set_selected.difference(set_result), columns=["not_in_youtube_result"])
df_not_in_result

In [ ]:
not_in_result_num = len(df_not_in_result)
not_in_result_num

In [ ]:
#df_not_in_result.to_excel(f"{lang_folder.capitalize()}_{not_in_result_num}_{file_ext}_\
#Not_In_{sample_num}_Youtube_{time_shift}s_Timeshift_Result.xlsx", index=False) 

In [ ]:
search_list2 = df_not_in_result["not_in_youtube_result"].to_list()

In [ ]:
df_search_result2 = word_group_youtube(df_youtube_sentence, search_list2, "sentence", sample_num)
df_search_result2

In [ ]:
search_result2_num = df_search_result2["search_string"].nunique()
search_result2_num

In [ ]:
df_word_group_time_loc_result2 = word_group_time_loc(df_search_result2, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_result2

In [ ]:
df_word_group_time_loc_result2.start_time = df_word_group_time_loc_result2.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc_result2.end_time = df_word_group_time_loc_result2.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc_result2

In [ ]:
df_word_group_time_loc_result2["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_result2['video_id'].map(str)+"&t="+df_word_group_time_loc_result2['start_time'].map(str)+"s"
df_word_group_time_loc_result2

In [ ]:
search_result_num2 = df_word_group_time_loc_result2["search_string"].nunique()
search_result_num2

In [ ]:
df_word_group_time_loc_result2.to_excel(f"{lang_folder.capitalize()}_{search_result_num2}_{file_ext}_\
{sample_num}_Youtube_{time_shift}s_Timeshift_Result2.xlsx", index=False) 